# TensorFlow Serving

## Docker example

### Installazione

_docker pull tensorflow/serving_

In [81]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import json
import time

from random import randint

### Preparazione fake data e del modello

In [2]:
X1 = [randint(0,100) for i in range(1000)]
X2 = [randint(-50,50) for i in range(1000)]

In [3]:
df = pd.DataFrame({"x1" : X1, "x2" : X2})

In [4]:
def get_y(x):
    
    y = 2*x[0] + x[1]/2
    
    return y

In [5]:
df

,x1,x2
0,97,-35
1,93,20
2,8,4
3,23,19
4,83,-33
...,...,...
995,94,-27
996,22,-8
997,7,-21
998,71,-40


In [6]:
df['y'] = df.apply(get_y, axis=1)

In [7]:
df.head()

,x1,x2,y
0,97,-35,176.5
1,93,20,196.0
2,8,4,18.0
3,23,19,55.5
4,83,-33,149.5


In [8]:
X = df[['x1', 'x2']].values
y = df.y.values

In [98]:
with tf.device('/cpu:0'):
    model = keras.models.Sequential()

    model.add(keras.layers.Dense(100, input_shape=(2,)))
    model.add(keras.layers.Dense(1))

    model.compile(loss='mae', optimizer='adam', metrics='mae')
    
    model.fit(X,y, epochs=100, verbose=0)

In [99]:
with tf.device('/cpu:0'):
    y_hat = model.predict(X[0].reshape(-1,2).tolist())

In [100]:
y_hat

array([[176.3245]], dtype=float32)

### Salvataggio del modello

In [101]:
# Nome del modello
name = 'example'

# Versione del modello (TensorFlow Serving gestisce automaticamente l'ultima versione)
version = int(time.time())

# Salvataggio
model.save(filepath=f'models/{name}/{version}', save_format='pb', overwrite=True)
# model.save(filepath=f'models/{name}/{version}')

INFO:tensorflow:Assets written to: models/example/1676734038/assets


**saved_model_cli show --dir models/example/001 --tag_set serve --signature_def serving_default**

In [24]:
print("""The given SavedModel SignatureDef contains the following input(s):
  inputs['dense_2_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: serving_default_dense_2_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_3'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict""")

The given SavedModel SignatureDef contains the following input(s):
  inputs['dense_2_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: serving_default_dense_2_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_3'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


### Avvio del modello attraverso un'istanza docker

**path dei modelli**

_MODEL_DIR="$(pwd)/models"_

All'interno di questa cartella ci sarà dunque ogni singolo modello in una cartella (example in questo caso), all'interno di ogni cartella ci sarà dunque una sottocartella numerica che conterrà la versione del modello.

**avvio dell'istanza docker per il modello _example_**

docker run -t --rm -p 8501:8501 -v "$MODEL_DIR/example:/models/example" -e MODEL_NAME=example tensorflow/serving &

**esempi di interrogazione attraverso curl**

curl -d "{\"instances\": [[10]]}" -X POST http://localhost:8501/v1/models/example:predict

{
    "predictions": [[24.9380608]
    ]
}

curl -d "{\"instances\": [[10], [10], [20]]}" -X POST http://localhost:8501/v1/models/example:predict

### Interrogazione API attraverso la libreria "requests"

In [9]:
import requests

In [10]:
data = dict()

In [102]:
data = {"instances" : X[0].reshape(-1,2).tolist()}
data

{'instances': [[97, -35]]}

In [103]:
url = "http://localhost:8501/v1/models/example:predict"
# url = "http://localhost:8501/v1/models/example:classify"

In [104]:
response = requests.post(url, json=data)

In [105]:
response.status_code

200

In [106]:
response.json()

{'predictions': [[176.324509]]}